In [5]:
python -m pip install --upgrade pip
py -m pip install --upgrade openai --quiet
python -m pip install python-dotenv --quiet

SyntaxError: invalid syntax (2384772097.py, line 1)

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

ModuleNotFoundError: No module named 'openai'

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/get_response', methods=['POST']) 
def get_response():
    message = request.json['message']
    response = get_completion(message) 
    return {'response': response}